In [ ]:
# Core libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import shutil

# TensorFlow & Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [ ]:
# 🔹 Load Cats vs Dogs dataset
import tensorflow_datasets as tfds
import tensorflow as tf
import math

# Dataset constants
IMG_SIZE = 150
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset (80% train, 20% validation)
(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,  # returns (image, label)
    with_info=True
)

# 🔹 Preprocessing + Augmentation for training
def preprocess_and_augment(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    # Data augmentation
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = image / 255.0
    return image, label

# 🔹 Preprocessing for validation (no augmentation)
def preprocess_only(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

# 🔹 Prepare datasets for training
train_ds = (train_ds
            .map(preprocess_and_augment, num_parallel_calls=AUTOTUNE)
            .shuffle(1000)
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTOTUNE))

val_ds = (val_ds
          .map(preprocess_only, num_parallel_calls=AUTOTUNE)
          .batch(BATCH_SIZE)
          .cache()
          .prefetch(AUTOTUNE))

# 🔹 Print dataset info and batch counts
num_train_batches = math.ceil(ds_info.splits['train'].num_examples * 0.8 / BATCH_SIZE)
num_val_batches = math.ceil(ds_info.splits['train'].num_examples * 0.2 / BATCH_SIZE)

print(ds_info)
print("Training batches:", num_train_batches)
print("Validation batches:", num_val_batches)

In [ ]:
import tensorflow_datasets as tfds

(train_ds, val_ds), info = tfds.load(
    "cats_vs_dogs",
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes
num_classes = info.features["label"].num_classes
print("Number of classes:", num_classes)

# Dataset sizes
print("Training examples:", info.splits['train'].num_examples * 0.8)
print("Validation examples:", info.splits['train'].num_examples * 0.2)

# Inspect few samples
for image, label in train_ds.take(2):
    print("Image shape:", image.shape, "Label:", label.numpy())

print("Class names:", info.features["label"].names)


In [ ]:
import tensorflow as tf

IMG_SIZE = 150
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# 🔹 Training preprocessing + augmentation
def preprocess_and_augment(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

# 🔹 Validation preprocessing (no augmentation)
def preprocess_only(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# 🔹 Apply preprocessing
train_ds_aug = (
    train_ds
    .map(preprocess_and_augment, num_parallel_calls=AUTOTUNE)
    .shuffle(1000)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(AUTOTUNE)
)

val_ds_preprocessed = (
    val_ds
    .map(preprocess_only, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(AUTOTUNE)
)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
import tensorflow as tf

# Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True
    ),
    reduce_lr
]

In [ ]:
history = model.fit(
    train_ds_aug,
    validation_data=val_ds_preprocessed,
    epochs=20,
    callbacks=callbacks
)

In [ ]:
# Plot training & validation accuracy and loss

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12,5))

# Accuracy Plot
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Loss Plot
plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get one batch from validation dataset
for images, labels in val_ds_preprocessed.take(1):
    break

# Make predictions
predictions = model.predict(images)

plt.figure(figsize=(12, 8))

for i in range(9):
    plt.subplot(3, 3, i + 1)

    img = images[i]
    label = labels[i].numpy()

    pred_prob = predictions[i][0]  # Probability of class 1 (Dog)
    pred_label = 1 if pred_prob > 0.5 else 0

    plt.imshow(img.numpy())
    plt.axis('off')

    true_class = "Dog" if label == 1 else "Cat"
    predicted_class = "Dog" if pred_label == 1 else "Cat"

    confidence = pred_prob if pred_label == 1 else 1 - pred_prob

    plt.title(f"True: {true_class}\nPred: {predicted_class}\nConf: {confidence:.2f}")

plt.tight_layout()
plt.show()

# Save final trained model
model.save("cats_vs_dogs_cnn_model.keras")

In [ ]:
model.save("cats_vs_dogs_cnn_model.keras")

In [ ]:
# 🔹 Upload an image and predict Cat or Dog
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Load model if not in memory
try:
    model
except NameError:
    from tensorflow.keras.models import load_model
    model = load_model("cats_vs_dogs_cnn_model.keras")

# Upload image
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    pred_prob = model.predict(img_array)[0][0]
    pred_label = "Dog" if pred_prob > 0.5 else "Cat"
    confidence = pred_prob if pred_label == "Dog" else 1 - pred_prob

    # Display image and prediction
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {pred_label}\nConfidence: {confidence:.2f}")
    plt.show()

In [ ]:
# 🔹 Upload an image and predict Cat or Dog
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Load model if not in memory
try:
    model
except NameError:
    from tensorflow.keras.models import load_model
    model = load_model("cats_vs_dogs_cnn_model.keras")

# Upload image
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    pred_prob = model.predict(img_array)[0][0]
    pred_label = "Dog" if pred_prob > 0.5 else "Cat"
    confidence = pred_prob if pred_label == "Dog" else 1 - pred_prob

    # Display image and prediction
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {pred_label}\nConfidence: {confidence:.2f}")
    plt.show()